<a href="https://colab.research.google.com/github/hand-e-fr/OpenHosta/blob/3.0.0_beta1/OpenHosta/tree/3.0.0_beta1/docs/SmokeTest_ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instal OpenHosta

In [ ]:
!pip install --upgrade -qqq uv

# If you need to test a pre release uncomment:
VERSION="@3.0.3"

!uv pip install -U \
    "git+https://github.com/hand-e-fr/OpenHosta.git$VERSION"

# Install Ollama

This is to run a local model and have zero dependancies to externa API provides.

In [ ]:
# This seems not to be accepted by google colab anymore. 
# Run it in a separtated terminal i refused
#!apt install -y screen
#!curl -fsSL https://ollama.com/install.sh | sh
#!screen -dmS ollama ollama serve

# So we just show how to install Ollama on linux
!curl -fsSL https://ollama.com/install.sh | sh

We test is ollama is available and how fast Qwen3 runs on it.

In [33]:
!ollama run gpt-oss:20b hello --verbose  2>&1 | grep -E ":"

We have a user: "hello". They greet. We should respond politely. Use friendly tone. It's a simple greeting. Probably ask how I can help.
total duration:       724.45224ms
load duration:        281.59815ms
prompt eval count:    68 token(s)
prompt eval duration: 71.126712ms
prompt eval rate:     956.04 tokens/s
eval count:           53 token(s)
eval duration:        370.899835ms
eval rate:            142.90 tokens/s


# Use OpenHosta

In [2]:

from OpenHosta import config

# You can replace with your own API (OpenAI chat/completion compatble)
config.DefaultModel.base_url = "http://localhost:11434/v1"
config.DefaultModel.model_name = "gpt-oss:20b"
config.DefaultModel.api_key = "not used by ollama local api"


**We test that the API is working with a simple call**

`ask()` makes a very simple call to the API without adding any hidden prompt.


In [4]:
from OpenHosta import ask

ask("Hello World!")

'Hello! 👋 How can I help you today?'

# Build a workflow agent

Let's see how to impement an agent with memory within an object

In [6]:
import asyncio
from dataclasses import dataclass
from OpenHosta.asynchrone import emulate, closure

from enum import Enum

class SubjectsInMemoryStore(Enum):
    FamilyMemberLinks = "FamilyMemberLinks"
    PeopleNames = "PeopleNames"
    Grocery = "Grocery"
    CalendarEvents = "CalendarEvents"
    NotValidSubject = "NotValidSubject"
    
class ActionType(Enum):
    INSERT="INSERT"
    UPDATE="UPDATE"
    DELETE="DELETE"
    SELECT="SELECT"
    
    
@dataclass
class UserDetails:
    FirstName: str
    LastName: str
    Age: int

class MyAgentWithMemory:
    """
    This agent records elements from the conversation only if they are related to specific subkects
    """
    
    CurrentUserId:str = None
    MemoryElements:dict = {}
    
    ChatLogs = []
    
    async def is_about_element_that_we_record(self, sentence: str) -> SubjectsInMemoryStore:
        """
        This function takes a sentenc and identify if there is element to record.
        
        We only record elements on subjects that we are allowed to.

        Args:
            sentence (str): a snippet of text from a conversation

        Returns:
            SubjectsInMemoryStore: In what category to store if we store something
        """
        # UserName variable and value will be worwarded to the LLM
        SpeakerName=self.MemoryElements.get(SubjectsInMemoryStore.PeopleNames, {}).get(self.CurrentUserId)
        return await emulate()
    
    async def detect_action_type(self, sentence:str) -> ActionType:
        """
        Identify the type of action that is implicitly requested in the sentence.
        
        The sentence is produced by us user of our system. Our system has a memory.
        What action should be do on our memory to best answer user expectations?
        
        Return:
            ActionType
        """
        return await emulate() 
        
    async def is_telling_who_he_is(self, sentence:str) -> bool:
        """
        The speaker is telling about who he is in this sentence.

        Args:
            sentence (str): a sentence made by a speaker

        Returns:
            bool: True if we learn about his name or forstname
        """
        return await emulate()
    
    async def fill_user_details(self, sentence:str, previouse_details:UserDetails)->UserDetails:
        """
        Identifies data fields from the sentence

        Args:
            sentence (_type_): what the user say
            previouse_details: What we new about this user
            
        Return:
            Filled UserDetails object for this user
        """
        return await emulate()
    
    
    async def answer_to(self, sentence:str)->str:
        """
        Proces the use input, handle memory, than answer.

        Args:
            sentence (str): what the client say

        Returns:
            str: what the agent answers
        """
        if self.CurrentUserId is None:
            tells_who_he_is = await self.is_telling_who_he_is(sentence)
            print(tells_who_he_is)
            if tells_who_he_is:
                who = await self.fill_user_details(sentence, None)
                self.CurrentUserId = "you"
                if SubjectsInMemoryStore.PeopleNames not in self.MemoryElements:
                    self.MemoryElements[SubjectsInMemoryStore.PeopleNames] = {}
                self.MemoryElements.get(SubjectsInMemoryStore.PeopleNames)[self.CurrentUserId] = who
                
                return await self.format_answer("user name recoded", who, sentence)
            else:
                return await self.format_answer("user shall identify first", None, sentence)
        else:
            return await self.process_question(sentence)
            
        
    async def find_first_element(self, subject:SubjectsInMemoryStore, question:str):
        """
        Find who we are speaking aabout

        Args:
            question (str): question
        """
        async def is_target(subject, question, key, value)->bool:
            """
            Decide if the suject that is refered to by the question is the one described by key:value.
            
            If it is clear that we speak about this one, return True.
            Otherwise return False.
            """
            return await emulate()
        
        for k,v in self.MemoryElements.items():
            print(f"Look: {k}, {v}")
            if await is_target(subject, question, k, v):
                print("FOUND: ", k, v)
                return k, v
            
        return None, None
            
    async def format_answer(self, instruction, data, question)->str:
        """
        Format a written answer to the question knowing that have executed `instruction` on 'data'.

        Args:
            instruction (_type_): what we have done or that we want to tell the user that we have done
            data (_type_): the data found, inserted or modified
            question (_type_): wht the used originally asked for.

        Returns:
            str: what we say to the user as an answer to his question
        """
        return await emulate()
        
        
    async def process_question(self, question):
        """
        This is the main logic for thiw workflow agent

        Args:
            question (str): user question
        """
        subject, action  = await asyncio.gather(
            self.is_about_element_that_we_record(question),
            self.detect_action_type(question)
        )
        print(subject, action)

        if subject is SubjectsInMemoryStore.NotValidSubject:
            return await self.format_answer("this suject is not handeled by this assistant", action, question)

        if action is ActionType.INSERT:
            if subject is SubjectsInMemoryStore.PeopleNames:      
                who_name, details = await asyncio.gather(
                    closure("return a name for the person that we shall remember")(question),
                    self.fill_user_details(question, None)
                )                
                data = self.MemoryElements.get(SubjectsInMemoryStore.PeopleNames, {})[who_name] = details
                return await self.format_answer("we have recorded the people", data, question )
                
            else:
                what_name, what_desc = await asyncio.gather(
                    closure("return a name for the element that we shall remember")(question),
                    closure("return a description of the element that we shall remember")(question)
                )
                if subject not in self.MemoryElements:
                    self.MemoryElements[subject] = {}
                self.MemoryElements.get(subject)[what_name] = what_desc
                return await self.format_answer("we have recorded the element", what_desc, question )
                
        elif action is ActionType.SELECT:
            key, data = await self.find_first_element(subject, question)
            
            if key is None:
                return await self.format_answer("we have not found the element in out knowledge", None, question )
                
            print("FOUND: ", key, data)
            return await self.format_answer("we have found the element in out knowledge", {"key":key, "data":data}, question)
            
        else:
            return await self.format_answer("action not yet supported", action, question)
            
        


In [7]:
A=MyAgentWithMemory()


In [12]:
await A.answer_to("bonjour, je suis emmanuel")

SubjectsInMemoryStore.PeopleNames ActionType.INSERT


'Bonjour Emmanuel, nous avons enregistré vos informations.'

In [13]:
await A.answer_to("Quelle sera la météo demain ?")

SubjectsInMemoryStore.NotValidSubject ActionType.SELECT


"Je suis désolé, mais ce sujet n'est pas pris en charge par cet assistant."

In [14]:
await A.answer_to("Qui es-tu ?")

SubjectsInMemoryStore.NotValidSubject ActionType.SELECT


'I’m sorry, I’m not able to answer that question.'

In [29]:
await A.answer_to("Qui suis-je ?")

SubjectsInMemoryStore.NotValidSubject ActionType.SELECT


'Je suis désolé, mais ce sujet ne relève pas de mes compétences.'

In [25]:
await A.answer_to("Je suis emmanuel batt. mémorise cela. je suis né en 1983")

SubjectsInMemoryStore.PeopleNames ActionType.INSERT


'I have recorded the details: First Name\u202f=\u202fEmmanuel, Last Name\u202f=\u202fBatt, Age\u202f=\u202f42.'

In [26]:
A.MemoryElements

{<SubjectsInMemoryStore.PeopleNames: 'PeopleNames'>: {'you': UserDetails(FirstName='Emmanuel', LastName='', Age=0),
  'Emmanuel': UserDetails(FirstName='emmanuel', LastName='', Age=0),
  'Emmanuel Batt': UserDetails(FirstName='emmanuel', LastName='batt', Age=42)},
 <SubjectsInMemoryStore.Grocery: 'Grocery'>: {'bread_purchase': "C'est une phrase indiquant la nécessité d'acheter du pain."}}

In [19]:
await A.answer_to("Il faut acheter du pain")

SubjectsInMemoryStore.Grocery ActionType.INSERT


'You asked, “Il faut acheter du pain.”  \nWe have recorded the following element: **C’est une phrase indiquant la nécessité d’acheter du pain.**'

In [20]:
await A.answer_to("Que faut il acheter ?")

SubjectsInMemoryStore.Grocery ActionType.SELECT
Look: SubjectsInMemoryStore.PeopleNames, {'you': UserDetails(FirstName='Emmanuel', LastName='', Age=0), 'Emmanuel': UserDetails(FirstName='emmanuel', LastName='', Age=0), 'Emmanuel Batt': UserDetails(FirstName='Emmanuel', LastName='Batt', Age=0)}
Look: SubjectsInMemoryStore.Grocery, {'bread_purchase': "C'est une phrase indiquant la nécessité d'acheter du pain."}
FOUND:  SubjectsInMemoryStore.Grocery {'bread_purchase': "C'est une phrase indiquant la nécessité d'acheter du pain."}
FOUND:  SubjectsInMemoryStore.Grocery {'bread_purchase': "C'est une phrase indiquant la nécessité d'acheter du pain."}


'Vous devez acheter du pain.'

In [30]:
await A.answer_to("Faut il acheter de l'eau ?")

SubjectsInMemoryStore.Grocery ActionType.SELECT
Look: SubjectsInMemoryStore.PeopleNames, {'you': UserDetails(FirstName='Emmanuel', LastName='', Age=0), 'Emmanuel': UserDetails(FirstName='emmanuel', LastName='', Age=0), 'Emmanuel Batt': UserDetails(FirstName='emmanuel', LastName='batt', Age=42)}
Look: SubjectsInMemoryStore.Grocery, {'bread_purchase': "C'est une phrase indiquant la nécessité d'acheter du pain."}


'I’m sorry, but I couldn’t find any information in my knowledge to answer whether you should buy water.'

In [31]:
await A.answer_to("Faut-il acheter du pain ?")

SubjectsInMemoryStore.Grocery ActionType.SELECT
Look: SubjectsInMemoryStore.PeopleNames, {'you': UserDetails(FirstName='Emmanuel', LastName='', Age=0), 'Emmanuel': UserDetails(FirstName='emmanuel', LastName='', Age=0), 'Emmanuel Batt': UserDetails(FirstName='emmanuel', LastName='batt', Age=42)}
Look: SubjectsInMemoryStore.Grocery, {'bread_purchase': "C'est une phrase indiquant la nécessité d'acheter du pain."}
FOUND:  SubjectsInMemoryStore.Grocery {'bread_purchase': "C'est une phrase indiquant la nécessité d'acheter du pain."}
FOUND:  SubjectsInMemoryStore.Grocery {'bread_purchase': "C'est une phrase indiquant la nécessité d'acheter du pain."}


'Oui, il faut acheter du pain.'

In [32]:
from OpenHosta import print_last_prompt
print_last_prompt(A.format_answer)

System prompt:
-----------------
You will act as a simulator for functions that cannot be implemented in actual code.

I'll provide you with function definitions described in Python syntax. 
These functions will have no body and may even be impossible to implement in real code, 
so do not attempt to generate the implementation.

Instead, imagine a realistic or reasonable output that matches the function description.
I'll ask questions by directly writing out function calls as one would call them in Python.
Respond with an appropriate return value, without adding any extra comments or explanations.
If the provided information isn't enough to determine a clear answer, respond simply with "None".
If assumptions need to be made, ensure they stay realistic, align with the provided description.

Here's the function definition:

```python
<class 'str'>

def format_answer(self, instruction, data, question) -> str:
    """
        Format a written answer to the question knowing that have execut